In [ ]:
import numpy as np
import h5py
import tensorflow as tf
import os
import pickle
import matplotlib.pyplot as plt
import random
from keras.preprocessing import sequence
import numpy as np
import h5py
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import MinMaxScaler , minmax_scale
from sklearn.preprocessing import StandardScaler
import time
import pandas as pd
new_MITmon_matrix=['FP1-F7','F7-T7','T7-FT9','T7-P7','P7-O1','FP1-F3','F3-C3','FZ-CZ','C3-P3','P3-O1',
                'FP2-F4', 'F4-C4','CZ-PZ','C4-P4','P4-O2','FP2-F8','F8-T8','FT10-T8','T8-P8','P8-O2' ]

Mon_matrix=MITmon_matrix7
Mon_matrix_col_no=4
Mon_matrix_row_no=5 #row는 4로 고정

sampling_rate=256 #MIT는 256, SNUH는 200
freq_seg=0.5 #freq domain 기본 간격freq_total_n=sampling_rate/freq_seg # freqdomain 간격 총 개수
freq_upper_range=40 #freq upper range Hz
freq_n=int(freq_upper_range/freq_seg) #일단 현재는 40

sec_pre_sz=40 #sz onset 전
sec_post_sz=40 # sz onset 후
sec_int_sz=sec_pre_sz+sec_post_sz

sec_frame=20 #LSTM의 frame 길이

ch_total_no = 20

power_weight=1 #가중치
with open('/MIT_data/source/pickle_file/information.pickle', 'rb') as f:
    information_dict = pickle.load(f)

In [ ]:
Mon_matrix = new_MITmon_matrix 
new_index_list = []
new_Mon_matrix_col_no=4
new_Mon_matrix_row_no=5

for i in range(len(Mon_matrix)):  ## h5 파일에 data 집어넣을시 위치 지정할 list 생성         
    a=4-(i % new_Mon_matrix_row_no) ## i%new_Mon_matrix_row_no의 의미 i와 new_Mon_matrix_row_no를 나누었을때의 나머지
    b=int(i/new_Mon_matrix_row_no)
    new_index_list.append([a,b,new_MITmon_matrix[i]])

In [ ]:
keylist=list(information_dict.keys())
sz_dict = {}
for j in range(0,24):   
    sz_dict[j+1] = {}
    start_list = []
    sz_len_list = []
    #h5f=h5py.File('/MIT_data/source/h5_file/stft/'+'chb'+str(keylist[j])+'_'+'data.h5','r')
    #data=h5f['all_data/data']     
   
    for k in range(len(information_dict[keylist[j]]['new_sz_info_list'])): 
        sz_edf_num=information_dict[keylist[j]]['new_sz_info_list'][k][-1]       
        before_time = 0
        for ti in range(information_dict[keylist[j]]['new_sz_info_list'][k][-1]-1):
            before_time = before_time + information_dict[keylist[j]]['time'][ti]
        start = int(information_dict[keylist[j]]['new_sz_info_list'][k][0] + before_time)   
        #print(start)
        start_list.append(start)
        sz_len_list.append(information_dict[keylist[j]]['new_sz_info_list'][k][1]-information_dict[keylist[j]]['new_sz_info_list'][k][0])
    sz_dict[j+1]['start'] = start_list
    sz_dict[j+1]['len'] = sz_len_list

In [ ]:
freq_n,time,Mon_matrix_row_no,Mon_matrix_col_no

In [ ]:
h5f.close()

In [ ]:
for order in range(1,2): 
    total_length_sec = 0
    chb_sec = 0     
    total_length_sec  = 0
    total_length_sec=sum(information_dict[order]['time'])    
    h5f=h5py.File('/MIT_data/source/h5_file/stft_data/chb'+str(order)+'_data.h5', 'a') 
    h5f.create_group('all_data')
    h5f.create_dataset('all_data/data', (freq_n,total_length_sec,Mon_matrix_row_no,Mon_matrix_col_no), compression="lzf")#   
    len_score=0  
    pickle_list=os.listdir('/MIT_data/source/pickle_file/stft_filter/'+str(order))##각 환자별 pickle list 
    for x in range(0,len(pickle_list)):## 딥러닝 모델에 맞게 데이터 형식 변형         
        with open('/MIT_data/source/pickle_file/stft_filter/'+str(order)+'/'+pickle_list[x], 'rb') as f:
            data = pickle.load(f)
        len_=len(data[MITmon_matrix7[1]][0])        
        time=len_  # EDF 파일 전체에서 FFT 하고 남는 마지막 하나 수치를 빼기 위해.   

        channel_location_data=[]
        try_=np.zeros((freq_n,time,Mon_matrix_row_no,Mon_matrix_col_no))
        input_zero = np.zeros((freq_n,time))           
        score = 0
        for i in range(len(new_index_list)):
            a=new_index_list[i][0]
            b=new_index_list[i][1]
            if new_index_list[i][-1] == -1:
                try_[:,:,a,b] = input_zero
            else:                
                try:
                    try_[:,:,a,b] = data[new_index_list[i][-1]]
                except:
                    try_[:,:,a,b] = data[new_index_list[i][-1]][:,:-1] 
                            

        channel_location_data = np.array(try_)               
        len_score = len_score + len(channel_location_data[0]) ## 반복문이 돌아가면서 들어갈 위치 지정       

                       
        h5f['all_data/data'][:,len_score-len(channel_location_data[0]):len_score,:] = channel_location_data 
       
          

        print('len_score:'+str(len_score))      
       
    h5f.close()                
                
    

In [ ]:
## label 부분 만드는 코드
for order in range(1,2): 
    pickle_list=os.listdir('pickle_file/stft_filter/'+str(order))##각 환자별 pickle list 
    len_score = 0
    h5f=h5py.File('h5_file/label_data_/chb'+str(order)+'_data.h5', 'a') 
    h5f.create_group('all_data')
    h5f.create_dataset('all_data/label', (total_length_sec,2), compression="lzf") ## lable index 생성
    for x in range(0,len(pickle_list)):
        with open('pickle_file/stft_filter/'+str(order)+'/'+pickle_list[x], 'rb') as f:
            data = pickle.load(f)
        len_score = len_score + len(data['FP1-F7'][0])## label의 길이를 맞추기 위해 가져온다
        train_y = []    
        make_real = []   
        match_dict = {}
        for edf in range(len(information_dict[order]['new_sz_info_list'])):
            if pickle_list[x].split('.')[0] == str(information_dict[order]['new_sz_info_list'][edf][-1]).zfill(2):                
                ## 하나의 pickle 파일안에 두개 이상의 seizure이 있을 경우가 있기 때문에 match_dict에 key가 필요함
                match_dict[edf] = {}
                match_dict[edf] = edf       

        ## seizure 부분이 있는 경우
        if match_dict != {}:            
            match_key=list(match_dict.keys())
            print('-----------------------------')
            ## 지정된 길이만큼 0의 리스트를 만든다
            for i in range(0,len(data['FP1-F7'][0])):
                make_real.append(0)
            ## 만들어진 리스트에 seizure 부분에 1을 집어 넣는다    
            for k in range(len(match_dict)):                                 
                for i in range(information_dict[order]['new_sz_info_list'][match_key[k]][0],information_dict[order]['new_sz_info_list'][match_key[k]][1]):
                    make_real[i] =1     
            make_real = np.array(make_real)
            ## keras에 있는 모듈을 사용하여 category data를 만든다
            cate_y=to_categorical(make_real,num_classes=2)
            
            h5f['all_data/label'][len_score-len(cate_y):len_score]= cate_y
        ## seizure 부분이 없는 경우
        else:          
            ## 모든 부분을 0을 채우고 그대로 진행
            for i in range(0,len(data['FP1-F7'][0])):
                make_real.append(0)                      
            make_real = np.array(make_real)
            
            ## keras에 있는 모듈을 사용하여 category data를 만든다
            cate_y=to_categorical(make_real,num_classes=2)


            h5f['all_data/label'][len_score-len(cate_y):len_score]= cate_y## 지정된 위치에 데이터를 집어넣는다

        print('len_score:'+str(len_score))      ## 들어가는 위치 확인  

    h5f.close() ## 들어가고 h5f 파일 손상을 막기위해 닫는다.                

In [ ]:
#se_dict = {}
weight_normal_dict= {}

for order in range(0,24):   
    num =  order + 1
    weight_normal_dict[num] = []
  
    #se_dict[num] = {}
    print(num)    
    #h5f=h5py.File('h5_file/stft_filter_60/'+'chb'+str(num)+'_'+'data.h5','r')
    label=h5py.File('h5_file/label_data/'+'chb'+str(num)+'_'+'data.h5','r')
    total_length_sec = len(label['all_data/label']) 
    for i in range(total_length_sec):
        if label['all_data/label'][i][0] == 0:
            weight_normal_dict[num].append(se_dict[num][0]/sum(se_dict[num]) * 100)
           
        else:            
            weight_normal_dict[num].append(se_dict[num][1]/sum(se_dict[num]) * 100)
    with open('/MIT_data/source/pickle_file/weight_dict.pickle', 'wb') as f:
        pickle.dump(freq_dict, f, pickle.HIGHEST_PROTOCOL)    